# Experiments for FISTA restart


First experiment:
$$\min\limits_{x\in R^N} \frac{1}{2}\|Ax-b\|^2_2+\rho\|x\|_1.$$

In [ ]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import time

In [ ]:
def F(x,A,b,rho):
    return 1/2*npl.norm(np.dot(A,x)-b)**2+rho*npl.norm(x,1)

def Df(x,A,b):
    return A.T.dot(np.dot(A,x)-b)

def proxh(x,rho,s):
    sign=x/np.absolute(x)
    sign[np.where(np.isnan(sign))]=0
    temp=np.absolute(x)-s*rho
    temp=temp*(temp>0)
    return sign*temp

In [ ]:
def ForwardBackward_step(x,s,A,b,rho):
    return proxh(x-s*Df(x,A,b),rho,s)

In [ ]:
def ForwardBackward(x,s,A,b,rho,Niter,epsilon):
    cout=[F(x,A,b,rho)]
    i=0
    out=False
    while i<Niter and out==False:
        xm=np.copy(x)
        x=ForwardBackward_step(x,s,A,b,rho)
        out=npl.norm(xm-x,2)<epsilon
        cout+=[F(x,A,b,rho)]
        i+=1
    return x,cout

In [ ]:
def FISTA(x,s,alpha,A,b,rho,Niter,epsilon,opt=False):
    if opt==False:
        cout=[F(x,A,b,rho)]
    else:
        cout=[]
    y=np.copy(x)
    i=0
    out=False
    while i<Niter and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(y,s,A,b,rho)
        out=npl.norm(y-x,2)<epsilon
        y=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,A,b,rho)]
    return x,cout,out

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)

In [ ]:
def FISTA_alamo(x,s,alpha,A,b,rho,Niter,epsilon):
    cout=[F(x,A,b,rho)]
    y=np.copy(x)
    i=0
    Ec=False
    out=False
    while (Ec==False or i<Niter) and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(y,s,A,b,rho)
        out=npl.norm(y-x,2)<epsilon
        y=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,A,b,rho)]
        m=int(i/2)+1
        Ec=((np.exp(1)*(cout[m]-cout[i])<=(cout[0]-cout[m])) and (cout[i]<=cout[0]))
    return x,cout,out


def restart_FISTA_alamo(x,s,alpha,A,b,rho,Niter,k,epsilon):
    i=0
    cout=[F(x,A,b,rho)]
    xm=x
    Fx=cout[0]
    Fxm=Fx
    out=False
    while (i<Niter) and out==False:
        xmm=xm
        xm=x
        Fxmm=Fxm
        Fxm=Fx
        x=ForwardBackward_step(x,s,A,b,rho)
        x,cout1,out=FISTA_alamo(x,s,alpha,A,b,rho,np.minimum(k,Niter-i-1),epsilon)
        cout=np.concatenate((cout,cout1))
        Fx=cout[-1]
        km=k
        k=len(cout1)-1
        i+=k+1
        if (np.exp(1)*(Fxm-Fx)>(Fxmm-Fxm))and (Fxmm!=Fxm):
            k=2*km
    return np.array(x),np.array(cout)

In [ ]:
def restart_FISTA_adap(x,s,alpha,A,b,rho,Niter,C,epsilon):
    i=0
    k=int(2*C)
    cout=[F(x,A,b,rho)]
    F_tab=[cout[-1]]
    k_tab=[k]
    mu_tab=[]
    x,cout_temp,out=FISTA(x,s,alpha,A,b,rho,k,epsilon)
    cout=np.concatenate((cout,cout_temp))
    i+=k
    F_tab+=[cout[-1]]
    k_tab+=[k]
    while (i<Niter) and out==False:
        x,cout_temp,out=FISTA(x,s,alpha,A,b,rho,np.minimum(k,Niter-i),epsilon)
        cout=np.concatenate((cout,cout_temp))
        i+=len(cout_temp)
        F_tab+=[cout[-1]]
        F_array=np.array(F_tab)
        ### Calcul de l'estimation de mu
        tab=4/(s*(np.array(k_tab)[:-1]+1)**2)*(F_array[:-2]-F_array[-1])/(F_array[1:-1]-F_array[-1])
        mu_tab+=[np.min(tab)]
        ### Ajustement de n_j en fonction 
        if (k<=C/np.sqrt(s*mu_tab[-1])):
            k=2*k
        k_tab+=[k]
    return x,cout

In [ ]:
def restart_grad_alamo(x0,s,alpha,A,b,rho,Niter,L,epsilon):
    cout=[F(x0,A,b,rho)]
    x=ForwardBackward_step(x0,s,A,b,rho)
    cout+=[F(x,A,b,rho)]
    y=np.copy(x)
    gamma=L*npl.norm(x0-x,2)
    i=0
    k=0
    out=False
    xp=ForwardBackward_step(y,s,A,b,rho)
    while i<Niter and out==False:
        i+=1
        k+=1
        xm=np.copy(x)
        x=np.copy(xp)
        out=npl.norm(y-x,2)<epsilon
        cout+=[F(x,A,b,rho)]
        y=x+(k-1)/(k+alpha-1)*(x-xm)
        xp=ForwardBackward_step(y,s,A,b,rho)
        temp=L*npl.norm(y-xp,2)
        if temp<=gamma*np.exp(-1):
            gamma=temp
            k=0
            i+=1
            xm=np.copy(x)
            x=np.copy(xp)
            xp=ForwardBackward_step(x,s,A,b,rho)
            cout+=[F(x,A,b,rho)]
            
    return x,cout

In [ ]:
def FISTA_restart_emp_grad(x,s,alpha,A,b,rho,Niter,epsilon):
    cout=[F(x,A,b,rho)]
    y=np.copy(x)
    i=0
    verif=False
    out=False
    while i<Niter and verif==False and out==False:
        i+=1
        xm=np.copy(x)
        ym=np.copy(y)
        x=ForwardBackward_step(y,s,A,b,rho)
        out=npl.norm(y-x,2)<epsilon
        y=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,A,b,rho)]
        verif=np.dot(ym-x,x-xm)>0
    return x,cout,out


def restart_emp_grad(x0,s,alpha,A,b,rho,Niter,epsilon):
    i=0
    cout=[]
    x=x0
    out=False
    while (i<Niter) and out==False:
        x,cout1,out=FISTA_restart_emp_grad(x,s,alpha,A,b,rho,Niter-i,epsilon)
        ni=len(cout1)
        i+=ni
        cout=np.concatenate((cout,cout1))
    return x,cout

In [ ]:
def FISTA_restart_emp_f(x,s,alpha,A,b,rho,Niter,epsilon):
    cout=[F(x,A,b,rho)]
    y=np.copy(x)
    i=0
    verif=False
    out=False
    while i<Niter and verif==False and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(y,s,A,b,rho)
        out=npl.norm(y-x,2)<epsilon
        y=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,A,b,rho)]
        verif=cout[-1]>cout[-2]
    return x,cout,out


def restart_emp_f(x0,s,alpha,A,b,rho,Niter,epsilon):
    i=0
    cout=[]
    x=x0
    out=False
    while (i<Niter) and out==False:
        x,cout1,out=FISTA_restart_emp_f(x,s,alpha,A,b,rho,Niter-i,epsilon)
        ni=len(cout1)
        i+=ni
        cout=np.concatenate((cout,cout1))
    return x,cout

In [ ]:
axis_font={'size':'18'}

In [ ]:
N=2000
niter=60000
A=10*np.random.rand(N,N)
b=10*np.random.rand(N)
rho=10
alpha=3
vp=np.absolute(npl.eig(A.T.dot(A))[0])
L=np.max(vp)
x=(np.random.rand(N)-0.5)*10
epsilon=1e-9


x_alamo,cout_alamo=restart_FISTA_alamo(x,1/L,alpha,A,b,rho,niter,1,epsilon)
x_adap,cout_adap=restart_FISTA_adap(x,1/L,alpha,A,b,rho,niter,6.38,epsilon)
x_grad,cout_grad=restart_grad_alamo(x,1/L,alpha,A,b,rho,niter,L,epsilon)
x_grad_emp,cout_grad_emp=restart_emp_grad(x,1/L,alpha,A,b,rho,niter,epsilon)
x_f_emp,cout_f_emp=restart_emp_f(x,1/L,alpha,A,b,rho,niter,epsilon)

slow_niter=int(np.max([len(cout_alamo),len(cout_adap),len(cout_grad),len(cout_grad_emp),len(cout_f_emp)])*2)
x_FB,cout_FB=ForwardBackward(x,1/L,A,b,rho,slow_niter,epsilon)
x_FISTA,cout_FISTA,out=FISTA(x,1/L,alpha,A,b,rho,slow_niter,epsilon)



temp=(1-1e-10)
min_tot=np.min([np.min(cout_alamo),np.min(cout_FISTA),np.min(cout_adap),np.min(cout_grad),np.min(cout_grad_emp),np.min(cout_f_emp)])
plt.plot(np.log(cout_FB-temp*min_tot))
plt.plot(np.log(cout_FISTA-temp*min_tot),'-')
plt.plot(np.log(cout_f_emp-temp*min_tot),':')
plt.plot(np.log(cout_grad_emp-temp*min_tot))
plt.plot(np.log(cout_alamo-temp*min_tot),'--')
plt.plot(np.log(cout_grad-temp*min_tot),'-.')
plt.plot(np.log(cout_adap-temp*min_tot))
plt.legend(["Forward-Backward","FISTA","Empirical restart scheme 1","Empirical restart scheme 2","Restart scheme 1 by Alamo et al.","Restart scheme 2 by Alamo et al.","Algorithm 2, $C=6.38$"])
plt.xlabel("Number of iterations",**axis_font)
plt.ylabel("$\log(F(x_n)-F^*)$",**axis_font)